# 1. The Link between CO2 Emissions and Population 

The Kaya Identity presents the decomposition of CO2 emissions into four driving factors:  
- Population
- GDP per capita
- Energy Intensity
- Carbon Footprint of the Energy

Here we want to dig in to the Kaya Identity data and investigate the relationship between CO2 emissions and population. The main proposed questions are:  
- Is there a link between the **CO2 emissions** of a given country and its **population**?
- Are there differences across different countries and regions, is there a trend?

In [1]:
# Import necessary packages with common alias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import modules containing functions
from Functions import subset, plots

ModuleNotFoundError: No module named 'Functions'

`s3fs` is a PyFilesystem interface to Amazon S3 cloud storage. Here we use this module to retrieve data from S3 buckets.  
The `World_Population.xlsx` dataset was downloaded from [population.un.org](https://population.un.org/wpp/Download/Standard/Population/)

In [9]:
import s3fs

# Retrieve data from S3 buckets
co2_file = pd.ExcelFile('s3://s3groupbrugge/Data/CO2Highlights2020.xls')
population_file = pd.ExcelFile('s3://s3groupbrugge/Data/World_Population.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

# 2. Data Cleaning

In `World_Population.xlsx`, we can find many tables describing CO2 emissions in multi-level. Here we choose the `KAYA`table which contains statistics of the KAYA decomposition. 

This table shows the **relative changes** of CO2 emissions and population of each country/region from 1971 to 2018. It facilitates the analysis of CO2-population relationship in one specific country but not suitable for comparison in between.

The original dataset is poorly structured and has some features that are not conducive to data selection and manipulation, thus it's crucial to clean and standardize the data. Firstly, we should select a subset of the data and deal with the redundant or missing column names.

In [ ]:
# Create a list of columns to select from the table
list_of_cols = [i for i in range(50)]

# Create a list of years
year = [i for i in range(1971,2019)]

# Set the name of the selected columns
col_names = ['Name', 'Factor'] + year

# Parse the 'KAYA' table
kaya_co2 = co2_file.parse('KAYA', skiprows=[0,1,2,4], usecols= list_of_cols, names=col_names)

kaya_co2.head(10)

By looking at a random sample of the dataset rows, we can find: 
- multiple gap/duplicated rows 
- missing values of `str` type stored as ".."
- incomplete country/region values.

In [ ]:
# Drop all the gap rows
kaya_co2.dropna(how='all', axis=0, inplace=True)

# Drop all the duplicated rows
kaya_co2.drop_duplicates(inplace=True)

# Fill in the missing 'Name' values with forward fill method
kaya_co2['Name'].fillna(method="ffill", inplace=True)

# Replace all the ".." with real missing values
kaya_co2.replace("..", np.nan, inplace=True)

kaya_co2.head(10)

# 3. The link between the CO2-Population of a given country

Here we are interested in the relationship between the CO2 emissions and the population of a given country. We choose China's data as our research object. By specifying the values of `Name` and `Factor` columns, we retrieve the CO2-Pop data of China from 1971 to 2018. 

For the convenience of plotting, we transpose the subset so that the CO2 and population are in columns.


In [ ]:
# Subset of China's CO2-Pop
cp_china = kaya_co2[(kaya_co2['Name']=="People's Rep. of China")&(kaya_co2['Factor'].isin(["CO2 emissions", "Population"]))]

# Transpose the subset of certain ranges
year_range = [i for i in range(1971, 2019)]
cp_china_trans = cp_china[year_range].T

# Show the transposed dataset
print(cp_china_trans.head())
print("\nThe shape of the dataset after transpose:" + str(cp_china_trans.shape))

To explore the link between the CO2 emissions and the population of China, we plot these two variables as pairs using `matplotlib.pyplot`. Though the data was recorded in time-series, it doesn't necessarily indicate that the plot would be continuous. Therefore, a scatter plot would be a good choice, with the CO2 emissions represented along the x-axis and the population along the y-axis. This will allow us to easily spot any trends. Furthermore, we can add a regression line to elaborate on that.

**Note** that both CO2 emissions and population are recorded as relative level compared to the reference year(Normally 1900 at the level of 100). So while plotting, we subtract all values with 100 to better show the fold changes.

In [ ]:
# Set the style of the figure
plt.style.use("ggplot")

# Initalize a figure and axes
fig, ax = plt.subplots()
fig.set_size_inches([6,5])

# Draw a scatter plot and set the labels and the title
ax.scatter(cp_china_trans.iloc[:,0]-100, cp_china_trans.iloc[:,1]-100, color='steelblue')
ax.set_ylabel("Population", color="steelblue")
ax.set_xlabel("CO2 Emissions",color="steelblue")
ax.set_title("China Population vs CO2 Emissions", color="brown")

# Add a regression line to the plot
x = pd.Series.to_numpy(cp_china_trans.iloc[:,0]-100)
y = pd.Series.to_numpy(cp_china_trans.iloc[:,1]-100)
m, b = np.polyfit(x.astype(float), y.astype(float), 1)
ax.plot(x, m*x+b, color = 'salmon')

# Show the plot
plt.show()

# 4. Trends in the world and 5 continents

Now we want to answer the second proposed question: Are there differences across different countries and regions, is there a trend?

By looking at only one specific country, we can't conclude that this positive correlation between these two variables is a universal property among the world. So we extend our research into wider ranges: the world and 5 continents. Now that there are many repetitive codes to write, we can aggregate those code lines into functions. Several functions were made: 
- In module `subset`: 
> `transpose_T(df)`: Return the transpose of the given DataFrame from kaya co2 data.  
> `specify_df_T(df, name, factors)`: Get the transposed DataFrame of the given one specific region/country and factors.
- In modele `plots`:
> `ax_scatter(axes, x, y, xlabel, ylabel, title, color='steelblue', reg=False)`: Create a scatter plot on the given axes with certain parameters. The `reg` is `False` in default. A regression line will be added if it is set to `True`.

In [ ]:
# Specify the subsets with region and factors
cp_world = subset.specify_df_T(kaya_co2, "World", ["CO2 emissions", "Population"])
cp_africa = subset.specify_df_T(kaya_co2, "Africa", ["CO2 emissions", "Population"])
cp_americas = subset.specify_df_T(kaya_co2, "Americas", ["CO2 emissions", "Population"])
cp_asia = subset.specify_df_T(kaya_co2, "Asia", ["CO2 emissions", "Population"])
cp_europe = subset.specify_df_T(kaya_co2, "Europe", ["CO2 emissions", "Population"])
cp_oceania = subset.specify_df_T(kaya_co2, "Oceania", ["CO2 emissions", "Population"])

cp_world

#### Plot the CO2 emissions against population in 2×3 subplots

In [ ]:
# Create variables for data slicing
co2_x = "CO2 Emissions"
pop_y = "Population"

# Create a 2×3 subplots
fig, ax = plt.subplots(2,3)
fig.set_size_inches([10,7])

# Plot the world and continents data in 6 axes
plots.ax_scatter(ax[0,0], cp_world.iloc[:,0]-100, cp_world.iloc[:,1]-100, None, pop_y, "World", reg=True)
plots.ax_scatter(ax[0,1], cp_africa.iloc[:,0]-100, cp_africa.iloc[:,1]-100, None, None, "Africa", reg=True)
plots.ax_scatter(ax[0,2], cp_americas.iloc[:,0]-100, cp_americas.iloc[:,1]-100, None, None, "Americas", reg=True)
plots.ax_scatter(ax[1,0], cp_asia.iloc[:,0]-100, cp_asia.iloc[:,1]-100, co2_x, pop_y, "Asia", reg=True)
plots.ax_scatter(ax[1,1], cp_europe.iloc[:,0]-100, cp_europe.iloc[:,1]-100, co2_x, None, "Europe", reg=True)
plots.ax_scatter(ax[1,2], cp_oceania.iloc[:,0]-100, cp_oceania.iloc[:,1]-100, co2_x, None, "Oceania", reg=True)

# Show the plot
plt.show()

It is clear that in the above figures the population and co2 emissions are nicely positive-correlated except for **Europe**, and the linear regression lines fit pretty well, which suggests that this trend applies in most of the conditions. But why is **Europe** different from the other continents? We can look at the two variables in time-series separately.

In [ ]:
# Initalize a figure and axes
fig, ax = plt.subplots()

# Plot the CO2 emissions in salmon
ax.plot(cp_europe.index, cp_europe['CO2 emissions'], color='salmon', label='CO2')
ax.set_xlabel('Time')
ax.set_ylabel('CO2 emissions', color='salmon')
ax.tick_params('y', color='salmon')
ax.set_title('European CO2 and Pop in time-series', color='brown')

# Create a twin Axes that shares the x-axis
ax_twin = ax.twinx()

# Plot the population in steelblue
ax_twin.plot(cp_europe.index, cp_europe['Population'], color='steelblue', label='Pop')
ax_twin.set_ylabel('Population', color='steelblue')
ax_twin.tick_params('y', color='steelblue')

# Show the plot
plt.show()

# 5. Trends between different countries/regions

So far we have investigated the relation of CO2-pop on individual country/region bases, partially because the `KAYA` table is organized with relative level data. To explore the trend between different countries/regions, we need actual values so that they can be aggregated into one figure for comparison.

For this purpose, we turn to the `CO2 FC` table from the original `CO2Highlights2020.xls` file, and `ESTIMATES` data from UN's World Population Project. Initially we select the region and CO2/Population information in 2018 from two files. Then we apply the inner join to connect the two datasets. At last we plot the relations of two variables on the figure.

In [ ]:
# Create lists of column indexs and names
use_cols=[2, 5, 75]
names = ['Name', 'Type', '2018']

# Parse the ESTIMATES table
population_2018 = population_file.parse('ESTIMATES', skiprows=[i for i in range(16)], usecols=use_cols, names=names)

# Select the region and population subset
pop_2018 = population_2018.loc[population_2018['Type']=='Country/Area', ('Name','2018')]
pop_2018

In [ ]:
# Parse the CO2 FC table
co2_2018 = co2_file.parse("CO2 FC", skiprows=[i for i in range(23)], usecols=[0, 48], names=['Name', '2018'])

# Drop the rows with missing values
co2_2018.dropna(axis=0, inplace=True)

co2_2018

In [ ]:
# Inner join two datasets on "Region" values
co2_pop_2018 = co2_2018.merge(pop_2018, on='Name', suffixes=('_co2', '_pop'))

co2_pop_2018

Due to the huge disparity in quantity between countries/regions, it is better to use logarithmic scale on each axis. 

In [ ]:
# Initalize a figure and axes
fig, ax = plt.subplots()
fig.set_size_inches([6,6])

# Create a scatter plot on countris/regions
ax.scatter(co2_pop_2018['2018_pop'], co2_pop_2018['2018_co2'], color='steelblue')

# Set the scale to logarithmic
plt.xscale('log')
plt.yscale('log')

# Set the labels and title
ax.set_xlabel('Population [in thousands]', color='steelblue')
ax.set_ylabel('CO2 Emissions [in million tones]', color='steelblue')
ax.set_title('CO2 Emissions vs Population in 2018', color='brown')

# Show the plot
plt.show()